In [1]:
from mp_api.client import MPRester
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

API_KEY = "4IeHY5jVcrgiKXNuAo6Jgs7yC0Z3hsli"

with MPRester(API_KEY) as mpr:
    # Pull summary data
    materials = mpr.materials.insertion_electrodes.search(
            working_ion="Li",
            fields=[
                "battery_id", "battery_formula", "formula_discharge", "average_voltage"
                , "energy_grav", "capacity_grav", "stability_charge", "fracA_charge", "max_delta_volume"
            ]
        )

# Convert to DataFrame
mp_df = pd.DataFrame([{
    "battery_id": m.battery_id,
    "battery_formula": m.battery_formula,
    "average_voltage": m.average_voltage,
    "capacity_grav": m.capacity_grav,
    "energy_grav": m.energy_grav,
    "max_delta_volume": m.max_delta_volume,
    "fracA_charge": m.fracA_charge,
    "stability_charge": m.stability_charge,
} for m in materials])

# Drop any rows with missing values (critical)
mp_df = mp_df.dropna().reset_index(drop=True)

# Verify data exists
print("Number of materials pulled:", len(mp_df))
if len(mp_df) == 0:
    raise ValueError("No materials data available after dropping missing values!")

# -------------------------------
# Normalize and prepare features
# -------------------------------
features = ["average_voltage",
    "capacity_grav",
    "energy_grav",
    "max_delta_volume",
    "fracA_charge",
    "stability_charge"]
scaler = MinMaxScaler()
mp_df[features] = scaler.fit_transform(mp_df[features])

# Invert features where lower is better
for col in [
    "max_delta_volume",
    "fracA_charge",
    "stability_charge"]:
    mp_df[col] = 1 - mp_df[col]

# Compute composite intrinsic score
mp_df["intrinsic_score"] = mp_df[features].mean(axis=1)

# -------------------------------
# Rank and select top candidates
# -------------------------------
top_candidates = mp_df.sort_values("intrinsic_score", ascending=False).head(20)

# Print to check
print("Top candidates based on intrinsic score:")
print(top_candidates[["battery_id", "battery_formula", "intrinsic_score"]])


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Retrieving InsertionElectrodeDoc documents: 100%|██████████| 2774/2774 [00:00<00:00, 3421.30it/s] 

Number of materials pulled: 2774
Top candidates based on intrinsic score:
         battery_id battery_formula  intrinsic_score
421    mp-985585_Li        Li0-3ClO         0.805235
2384   mp-757276_Li  Li1-3Ti2(PO4)3         0.735347
4          mp-45_Li         Li0-3Ca         0.697137
40    mp-1111688_Li       Li0-3CrF6         0.691213
351    mp-759539_Li        Li0-3VF5         0.676689
338    mp-759184_Li        Li0-3VF5         0.676657
327    mp-758670_Li        Li0-3VF5         0.676333
69      mp-19308_Li       Li0-2NiO2         0.672207
122     mp-35925_Li       Li0-2NiO2         0.671213
211    mp-753115_Li       Li0-2MnF4         0.667002
151    mp-558059_Li       Li0-2MnF4         0.666936
116     mp-32686_Li       Li0-2CoO2         0.665155
265    mp-754868_Li       Li0-2FeO2         0.664041
68      mp-19279_Li       Li0-2MnO2         0.660819
579     mp-25323_Li    Li0-3Na2FeO4         0.657682
2636   mp-761177_Li       Li0-2VOF3         0.657358
2420   mp-757787_Li      

In [2]:
print(mpr.materials.insertion_electrodes.available_fields)

['builder_meta', 'battery_type', 'battery_id', 'thermo_type', 'battery_formula', 'working_ion', 'num_steps', 'max_voltage_step', 'last_updated', 'framework', 'framework_formula', 'elements', 'nelements', 'chemsys', 'formula_anonymous', 'warnings', 'formula_charge', 'formula_discharge', 'max_delta_volume', 'average_voltage', 'capacity_grav', 'capacity_vol', 'energy_grav', 'energy_vol', 'fracA_charge', 'fracA_discharge', 'stability_charge', 'stability_discharge', 'id_charge', 'id_discharge', 'host_structure', 'adj_pairs', 'material_ids', 'entries_composition_summary', 'electrode_object']


In [4]:
from mp_api.client import MPRester
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

API_KEY = "4IeHY5jVcrgiKXNuAo6Jgs7yC0Z3hsli"

with MPRester(API_KEY) as mpr:
    # Pull summary data
    materials = mpr.materials.insertion_electrodes.search(
            working_ion="Li",
            fields=[
                "battery_id", "battery_formula", "average_voltage"
            ]
        )


# Convert to DataFrame
mp_df = pd.DataFrame([{
    "battery_id": m.battery_id,
    "battery_formula": m.battery_formula,
    "average_voltage": m.average_voltage,

} for m in materials])

# Drop any rows with missing values (critical)
mp_df = mp_df.dropna().reset_index(drop=True)

# Verify data exists
print("Number of materials pulled:", len(mp_df))
if len(mp_df) == 0:
    raise ValueError("No materials data available after dropping missing values!")

# -------------------------------
# Normalize and prepare features
# -------------------------------
features = ["average_voltage"]
scaler = MinMaxScaler()
mp_df[features] = scaler.fit_transform(mp_df[features])

# Compute composite intrinsic score
mp_df["intrinsic_score"] = mp_df[features].mean(axis=1)

# -------------------------------
# Rank and select top candidates
# -------------------------------
top_candidates = mp_df.sort_values("intrinsic_score", ascending=False).head(20)


# Print to check
print("Top candidates based on intrinsic score:")
print(top_candidates[["battery_id", "battery_formula", "intrinsic_score"]])


Retrieving InsertionElectrodeDoc documents: 100%|██████████| 2774/2774 [00:00<00:00, 5188.22it/s] 

Number of materials pulled: 2774
Top candidates based on intrinsic score:
         battery_id    battery_formula  intrinsic_score
1610   mp-757276_Li     Li1-3Ti2(PO4)3         1.000000
1567   mp-756779_Li     Li0-0.25Fe2O3F         0.584961
2675   mp-807730_Li    Li0.5-1.5VP2HO8         0.570885
2166   mp-801838_Li      Li0-4V4(OF3)3         0.510491
180    mp-752527_Li    Li0.33-0.67FeO2         0.491195
2164   mp-799530_Li       Li0-3Mn5OF11         0.450210
2099   mp-776632_Li    Li0-1Mn4(P2O7)3         0.428557
468   mp-1189643_Li      Li0-1Co(CO3)2         0.418361
494   mp-1345203_Li       Li0-3Mn5OF11         0.417299
1133   mp-729639_Li     Li0-2Ni5(PO4)4         0.391693
1554   mp-756664_Li      Li0.67-1FeBO3         0.384981
390    mp-770684_Li        Li0-0.5CrO2         0.381654
791     mp-26449_Li        Li0-1NiP2O7         0.373100
1129   mp-705448_Li         Li0-1NiPO4         0.362815
203    mp-752973_Li          Li0-1MnF4         0.360300
2592   mp-771908_Li        Li0